In [2]:
import numpy as np
import pandas as pd

## Clean Recipe Ingredients
First, we want to clean the ingredients list into a better format since each recipe listed the ingredients differently. The best way to do this is to make a list of key ingredients and then search the list for these common ingredients that out model will be able recognize from the images input by the user. Then, find he items from that list in the recipe ingredients and create a new column in the dataframe to hold these key ingredients for each recipe.

In [3]:
df = pd.read_csv("/content/drive/MyDrive/data folder/recipes.csv")
df.head()

,Name,URL,Cuisine,Ingredients
0,Best Vinegar Coleslaw,https://www.allrecipes.com/recipe/59318/amish-...,Amish and Mennonite,"['1 large head cabbage, cored and finely shred..."
1,Pennsylvania Dutch Pickled Beets and Eggs,https://www.allrecipes.com/recipe/13743/pennsy...,Amish and Mennonite,"['8 large eggs', '2 (15 ounce) cans whole pick..."
2,Amish Macaroni Salad,https://www.allrecipes.com/recipe/74915/amish-...,Amish and Mennonite,"['2 cups uncooked elbow macaroni', '3 large ha..."
3,Amish Friendship Bread Starter,https://www.allrecipes.com/recipe/7063/amish-f...,Amish and Mennonite,"['1 (.25 ounce) package active dry yeast', '¼ ..."
4,My Amish Friend's Caramel Corn,https://www.allrecipes.com/recipe/74950/my-ami...,Amish and Mennonite,"['7 quarts plain popped popcorn', '2 cups dry ..."


Import the list of key ingredients and put them into a list of strings:

In [4]:
# Open the file in read mode
with open('/content/drive/MyDrive/data folder/ingredients.txt', 'r') as file:
    # Read the lines of the file
    lines = file.readlines()

# Strip newline characters to clean up the strings
# and create the list of strings
key_ingredients = [line.strip().lower() for line in lines]

print(key_ingredients)  # Output: ['apple', 'banana', 'orange', 'grape']

['artichoke', 'asparagus', 'broccoli', 'brussels sprouts', 'cabbage', 'cauliflower', 'celery', 'cilantro', 'eggplant', 'bok choy', 'lettuce', 'spinach', 'beans', 'chickpeas', 'green beans', 'lentils', 'peas', 'mushrooms', 'garlic', 'onion', 'bell pepper', 'hot pepper', 'beet', 'carrot', 'radish', 'turnip', 'potato', 'corn', 'squash', 'zucchini', 'cucumber', 'tomato', 'olives', 'rice', 'banana', 'apple', 'mango', 'grapes', 'cherries', 'rasberries', 'strawberries', 'blueberries', 'peach', 'plum', 'nectarine', 'apricot', 'blackberries', 'watermelon', 'pineapple', 'orange', 'avocado', 'lemon and lime', 'chicken', 'eggs', 'beef', 'pork', 'turkey', 'sausage', 'milk', 'cheese', 'butter', 'salmon', 'flour', 'sugar', 'chocolate']


The csv reads the list of ingredients as a string so we need to remove all punctuation and search the ingredients for the key ingredients.

In [5]:
import ast  # Library to handle string conversion to list
import string

def remove_punctuation(text):
  '''
  - Removes the punctuation from the string of recipe ingredients
  - Input: string of ingredients
  - Output: the cleaned string
  '''
  translator = str.maketrans('', '', string.punctuation)
  return text.translate(translator)

def find_key_ingredients(df, key_ingredients):
  '''
  - Compares the lists of ingredients and creates a new column in the dataframe
    with a list of key ingredients for each recipe
  - Input: dataframe of recipes, list of key ingredients
  - Output: dataframe with new column of key ingredients
  '''

  # Function to check if any key ingredient is found in a list of ingredients
  def find_word_in_string(text):
    '''
    - Finds key ingredients in ingredient string
    - Input: text from ingredient column of df
    - Output: List of key ingredients found in ingredient column
    '''
    R = []
    for i in key_ingredients:
      if i in text:
        # a lot of recipes use corn starch and corn flour
        if i == 'corn' and ('corn starch' in text or 'corn flour' in text):
          break;
        R.append(i)
        # Sometimes only need one egg
        if i == 'eggs' and 'egg' in text and 'eggs' not in R:
          R.append(i)
    return R

  # Remove punctuation
  df['Ingredients'] = df['Ingredients'].apply(remove_punctuation)

  # Find key ingredients in each row
  df['key_ingredients'] = df['Ingredients'].apply(find_word_in_string)
  return df

In [6]:
new_df = find_key_ingredients(df, key_ingredients)
new_df.head()

,Name,URL,Cuisine,Ingredients,key_ingredients
0,Best Vinegar Coleslaw,https://www.allrecipes.com/recipe/59318/amish-...,Amish and Mennonite,1 large head cabbage cored and finely shredded...,"[cabbage, celery, onion, sugar]"
1,Pennsylvania Dutch Pickled Beets and Eggs,https://www.allrecipes.com/recipe/13743/pennsy...,Amish and Mennonite,8 large eggs 2 15 ounce cans whole pickled bee...,"[onion, beet, eggs, sugar]"
2,Amish Macaroni Salad,https://www.allrecipes.com/recipe/74915/amish-...,Amish and Mennonite,2 cups uncooked elbow macaroni 3 large hardcoo...,"[celery, onion, bell pepper, eggs, sugar]"
3,Amish Friendship Bread Starter,https://www.allrecipes.com/recipe/7063/amish-f...,Amish and Mennonite,1 25 ounce package active dry yeast ¼ cup warm...,"[milk, flour, sugar]"
4,My Amish Friend's Caramel Corn,https://www.allrecipes.com/recipe/74950/my-ami...,Amish and Mennonite,7 quarts plain popped popcorn 2 cups dry roast...,"[corn, sugar]"


Upload new dataframe of recipes with key ingredients to a csv file:

In [7]:
new_df.to_csv('Recipes_cleaned.csv', index=False)

## Function to Generate Recipe Recommendations
Generate a list of recommended recipes based off of prefered cuisine and most matches of ingredients input.

In [8]:
def gen_recipe(cuisine, list_of_ingredients, df):
  '''
  - Generates recipe recommendations from the recipe dataset based off of the
    detected ingredients from user input images and prefered cuisine.
  - Input: cuisine, list of ingredients, dataframe of recipes
  - Output: dataframe of recomended recipes
  '''
  cuisine_df = df[df["Cuisine"] == cuisine].copy()
  # Create a new column to count matching ingredients
  cuisine_df['Matched Ingredients'] = cuisine_df['key_ingredients'].apply(lambda x: sum(1 for ingredient in list_of_ingredients if ingredient in x))
  cuisine_df = cuisine_df[cuisine_df['Matched Ingredients'] >= 2]
  # Sort dataframe based on the number of matched ingredients in descending order
  cuisine_df = cuisine_df.sort_values(by='Matched Ingredients', ascending=False)

  # Reset index
  cuisine_df.reset_index(drop=True, inplace=True)

  return cuisine_df

In [9]:
# Test 1
df_recs = gen_recipe('French', ['eggs', 'milk', 'butter'], new_df)
df_recs

,Name,URL,Cuisine,Ingredients,key_ingredients,Matched Ingredients
0,Basic Crêpes,https://www.allrecipes.com/recipe/16383/basic-...,French,1 cup allpurpose flour 2 large eggs ½ cup milk...,"[eggs, milk, butter, flour]",3
1,French Crêpes,https://www.allrecipes.com/recipe/20931/french...,French,1 cup allpurpose flour 1 teaspoon white sugar ...,"[eggs, milk, butter, flour, sugar]",3
2,Dessert Crepes,https://www.allrecipes.com/recipe/19037/desser...,French,4 eggs lightly beaten 1 ⅓ cups milk 1 cup all...,"[eggs, milk, butter, flour, sugar]",3
3,Pain au Chocolat (Chocolate Croissants),https://www.allrecipes.com/recipe/142764/pain-...,French,¼ cup warm water 2 ¼ cups bread flour 2 tables...,"[milk, butter, flour, sugar, chocolate]",2
4,Chef John's Crème Fraîche,https://www.allrecipes.com/recipe/222387/chef-...,French,2 cups heavy cream 3 tablespoons cultured butt...,"[milk, butter]",2
5,Pastry Cream,https://www.allrecipes.com/recipe/76043/pastry...,French,2 cups milk ¼ cup white sugar 2 egg yolks 1 ...,"[corn, milk, butter, sugar]",2
6,Creme Pat,https://www.allrecipes.com/creme-patissiere-74...,French,2 cups milk divided 2 tablespoons cornstarch 1...,"[corn, milk, butter, sugar]",2
7,Eclairs,https://www.allrecipes.com/recipe/19010/eclair...,French,1 cup water ½ cup butter 1 cup allpurpose flou...,"[eggs, butter, flour]",2
8,"Chicken French (Rochester, NY Style)",https://www.allrecipes.com/recipe/213487/chick...,French,¼ cup allpurpose flour or as needed salt and...,"[garlic, chicken, eggs, cheese, butter, flour,...",2
9,Delicious Easy Chicken Francese,https://www.allrecipes.com/recipe/16681/delici...,French,½ cup allpurpose flour 4 large eggs beaten 1 p...,"[corn, chicken, eggs, butter, flour]",2
